<a href="https://colab.research.google.com/github/Jeon-Gahui/BAF-Study-2026-winter/blob/main/RAG%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install cohere faiss-cpu rank_bm25 langchain-community langchain_huggingface

# 사용하는 파이썬과 CUDA 버전에 맞는 llama-cpp-python 패키지 설치
!pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.16-cu124/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl

### 생성모델 로드하기

In [ ]:
#생성 모델 로드
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf

In [19]:
from langchain_community.llms import LlamaCpp

# 런타임 : T4
llm = LlamaCpp(
    model_path = "/content/Phi-3-mini-4k-instruct-q4.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=4096,
    seed=42,
    verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64


### 임베딩 모델 로드하기

In [20]:
#sentense-transformers 새로 설치하면 오류남
from langchain_huggingface import HuggingFaceEmbeddings

# 텍스트를 수치 표현으로 변환하기 위한 임베딩 모델
embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-small-en-v1.5'
)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


###데이터셋 로드하기

In [ ]:
#datasets 패키지 설치
!pip install datasets

In [21]:
from datasets import load_dataset

# 'text-corpus' 설정을 사용하여 위키피디아 본문 데이터 가져오기
dataset = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus", split="passages")

# 데이터 구조 확인
print(dataset)
print(f"첫 번째 문서 내용: {dataset[0]['passage'][:100]}...")

Dataset({
    features: ['passage', 'id'],
    num_rows: 3200
})
첫 번째 문서 내용: Uruguay (official full name in  ; pron.  , Eastern Republic of  Uruguay) is a country located in the...


In [22]:
from langchain_core.documents import Document

# Hugging Face 데이터셋을 LangChain 문서 형식으로 변환
docs = [
    Document(page_content=item['passage'], metadata={"id": item['id']})
    for item in dataset
]

### DB 구축하기

In [ ]:
!pip install chromadb

In [23]:
from langchain_community.vectorstores import Chroma

#로컬 벡터 DB 생성 : Chroma
db = Chroma.from_documents(docs, embedding_model)

### 프롬프트 템플릿 생성

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA


# 프롬프트 템플릿을 만듭니다.
template = """<|user|>
Relevant information:
{context}

Provide a concise answer the following question using the relevant information provided above:
{question}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# RAG 파이프라인
rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=True
)

In [25]:
rag.invoke('The capital of America')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'The capital of America',
 'result': " The capital of the United States is Washington D.C., named after George Washington, and not directly related to James Monroe's influence on having a city named after him in Liberia; however, it was during his presidency that the American Colonization Society established Monrovia as a haven for freed slaves. The capital of Uruguay, Montevideo, has no direct connection to any U.S. President or their presidential influence."}

### 평가

In [26]:
# 평가 데이터셋 로드
eval_ds = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer", split="test")

# 5개 샘플만 테스트
for i in range(5):
    q = eval_ds[i]['question']
    real_a = eval_ds[i]['answer']

    response = rag.invoke(q)
    print(f"질문: {q}")
    print(f"실제 정답: {real_a}")
    print(f"모델 답변: {response['result']}\n")



> Entering new RetrievalQA chain...

> Finished chain.
질문: Was Abraham Lincoln the sixteenth President of the United States?
실제 정답: yes
모델 답변:  Yes, Abraham Lincoln was the sixteenth President of the United States. He served from March 4, 1861 until his assassination on April 15, 1865.



> Entering new RetrievalQA chain...

> Finished chain.
질문: Did Lincoln sign the National Banking Act of 1863?
실제 정답: yes
모델 답변:  Yes, Lincoln signed the National Banking Act of 1863 as part of other important legislation involving economic matters and the creation of a strong national financial system.



> Entering new RetrievalQA chain...

> Finished chain.
질문: Did his mother die of pneumonia?
실제 정답: no
모델 답변:  Based on the information provided, there is no mention of his mother dying of pneumonia.



> Entering new RetrievalQA chain...

> Finished chain.
질문: How many long was Lincoln's formal education?
실제 정답: 18 months
모델 답변:  Lincoln's formal education lasted for about 18 months.



> Entering 